In [208]:
import pandas as pd
import numpy as np
import random

In [209]:
def read_data(file):
    data = pd.read_excel(file, header=None)
    return data

In [210]:
data = read_data("JSSP-problems/basic-problem.xlsx")
data

,0,1,2
0,3,2,2
1,2,4,1
2,0,4,3


# CHROMOSOMA

Representat com una llista de tuples, on gada gen es una tupla. La primera posició de la tupla indica el job i la segona posició indica la màquina. 

In [211]:
params = {"population_size": 10, 
          "mutation_rate": 0.1, 
          "crossover_rate": 0.5, 
          "max_generations": 100}

In [212]:
def generate_inital_population(data, population_size):
    jobs = data.shape[0]
    machines = data.shape[1]
    population = []
    for _ in range(population_size):
        chromosome = []
        for job in range(jobs):
            for machine in range(machines):
                chromosome.append((job, machine))
        random.shuffle(chromosome)
        population.append(chromosome)
    return population


In [213]:
def fitness(chromosome, data):
    job_end_time = [0] * len(data)  # End time of each job
    machine_end_time = [0] * len(data[0])  # End time of each machine
    
    for task in chromosome:
        job, machine = map(int, task)
        start_time = max(job_end_time[job], machine_end_time[machine]) # Start time of the task
        job_end_time[job] = start_time + data[job][machine] # Update end time of job
        machine_end_time[machine] = start_time + data[job][machine] # Update end time of machine
    
    return max(machine_end_time)  # Makespan

In [214]:
def selection(population, fitness_values, tournament_size):
    selected = set()
    
    while len(selected) < 2:  # Select 2 parents
        tournament = random.sample(list(enumerate(fitness_values)), tournament_size)
        winner = min(tournament, key=lambda x: x[1])[0] # Select the chromosome with the lowest fitness value
        selected.add(winner)

    parent1 = population[selected[0]]
    parent2 = population[selected[1]]
    return parent1, parent2

In [215]:
def crossover(parent1, parent2, probability):
    size = len(parent1)
    child1, child2 = [], []
    for i in range(size):
        if random.random() < probability:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    return child1, child2

In [216]:
def mutation(chromosome, mutation_rate):
    pass

In [217]:
def gen_algorithm(data, params):
    population_size = params["population_size"]
    mutation_rate = params["mutation_rate"]
    crossover_rate = params["crossover_rate"]
    max_generations = params["max_generations"]
    tournament_size = 4

    population = generate_inital_population(data, population_size)
    fitness_values = [fitness(chromosome, data) for chromosome in population]
    print(fitness_values)
    for generation in range(max_generations):
        new_population = []
        for _ in range(population_size):
            parent1, parent2 = selection(population, fitness_values, tournament_size)
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            break
        break
    
    return population

In [218]:
gen_algorithm(data, params)

[13, 10, 12, 13, 18, 12, 12, 11, 19, 20]
[[(2, 0), (1, 2), (2, 1), (0, 0), (0, 2), (2, 2), (0, 1), (1, 1), (1, 0)], [(2, 1), (1, 2), (0, 2), (1, 1), (2, 0), (0, 0), (0, 1), (2, 2), (1, 0)], [(1, 0), (2, 2), (0, 1), (0, 2), (2, 0), (1, 2), (1, 1), (2, 1), (0, 0)], [(2, 0), (0, 0), (1, 2), (0, 2), (2, 2), (0, 1), (1, 1), (2, 1), (1, 0)], [(2, 0), (0, 0), (0, 1), (1, 1), (2, 1), (1, 2), (2, 2), (0, 2), (1, 0)], [(0, 0), (1, 1), (0, 1), (2, 2), (2, 0), (0, 2), (2, 1), (1, 2), (1, 0)], [(1, 2), (2, 0), (0, 2), (2, 1), (0, 1), (2, 2), (1, 1), (0, 0), (1, 0)], [(2, 0), (0, 2), (1, 2), (1, 1), (2, 2), (0, 0), (0, 1), (2, 1), (1, 0)], [(2, 1), (1, 1), (2, 0), (0, 1), (0, 0), (1, 0), (0, 2), (1, 2), (2, 2)], [(1, 0), (0, 0), (2, 0), (2, 2), (0, 2), (0, 1), (1, 1), (2, 1), (1, 2)]]
[6]


IndexError: list index out of range